In [1]:
import scipy.io as sio
import numpy as np
import datetime
import matplotlib as mpl
import tkinter
mpl.use('TkAgg')
import matplotlib.pyplot as plt
plt.ion()
# If using tkinter ('TkAgg') then make font color black,
mpl.rcParams.update({'text.color': 'k'})
mpl.rcParams.update({'axes.labelcolor': 'k'})
mpl.rcParams.update({'xtick.color': 'k'})
mpl.rcParams.update({'ytick.color': 'k'})
import numpy as np
import pandas as pd
#import sciencedates #same as igrf12fort
import spacepy.time as spt
#import xarray as xr #same as igrf12fort and sciencedates
mpl.rcParams.update({'font.size': 15})
mpl.rcParams.update({'figure.figsize' : [10.0, 8.0]})


# file = '/SPENCEdata/Research/Satellites/FAST/espec_identification/20181009-Kalle_inds_21-24MLT__BOTH__gigante_DB.sav'

# redMagFile = '/SPENCEdata/Research/Satellites/FAST/espec_identification/20181009-Kalle_inds_21-24MLT__BOTH__gigante_DB__MAG_coords.sav'

allFile='/SPENCEdata/Research/Satellites/FAST/espec_identification/20181009-Kalle_inds_21-24MLT__BOTH__gigante_DB__delta_t_and_mag_coords.sav'

# this = sio.readsav(file,python_dict=True)
# thisMAG = sio.readsav(redMagFile,python_dict=True)
all = sio.readsav(allFile,python_dict=True)

This unreleased version of SpacePy is not supported by the SpacePy team.


In [2]:
print(all.keys())
# print(thisMAG.keys())
espec = all['espec']
mag = all['mag']
dts = all['espec_delta_t']

dict_keys(['espec', 'good_i', 'espec_delta_t', 'mag'])


In [3]:
print(espec.dtype)
print(mag.dtype)
print(dts.dtype)
espec.x[0].shape
# espec.mono[0].shape
# espec.broad[0].shape
# print(dat.alt[0].shape)
# print(dat.lon[0].shape)
#print(dat.lat[0].shape)

(numpy.record, [(('x', 'X'), 'O'), (('orbit', 'ORBIT'), 'O'), (('mlt', 'MLT'), 'O'), (('ilat', 'ILAT'), 'O'), (('alt', 'ALT'), 'O'), (('mono', 'MONO'), 'O'), (('broad', 'BROAD'), 'O'), (('diffuse', 'DIFFUSE'), 'O'), (('je', 'JE'), 'O'), (('jee', 'JEE'), 'O'), (('nbad_espec', 'NBAD_ESPEC'), 'O'), (('mapfactor', 'MAPFACTOR'), 'O'), (('chare', 'CHARE'), 'O'), (('info', 'INFO'), 'O')])
(numpy.record, [(('alt', 'ALT'), 'O'), (('lon', 'LON'), 'O'), (('lat', 'LAT'), 'O')])
>f4


(4813389,)

In [33]:
# Get a datetime from our UTC doubles created by IDL
datetime.datetime.utcfromtimestamp(espec.x[0][-1])


datetime.datetime(2002, 10, 24, 23, 52, 22, 272221)

In [5]:
# Diffusers
diff_i=np.where(espec.diffuse[0] == 1)
print(diff_i)

# Uvver fings
# print(np.where((espec.mono[0] == 1) | (espec.mono[0] == 2))[0].shape)
# print(np.where(espec.diffuse[0] == 1)[0].shape)

(array([      0,       1,       2, ..., 4813386, 4813387, 4813388]),)


In [34]:
fig1 = plt.figure(num=1,figsize=(20, 8))
ax = fig1.add_subplot(111)
plt.hist(mag.lon[0])
plt.show()


(array([466869., 424997., 406490., 540650., 535440., 566529., 547119.,
        453908., 436565., 434822.]),
 array([7.3907075e-05, 3.6000057e+01, 7.2000038e+01, 1.0800002e+02,
        1.4400002e+02, 1.7999998e+02, 2.1599997e+02, 2.5199995e+02,
        2.8799994e+02, 3.2399994e+02, 3.5999991e+02], dtype=float32),
 <a list of 10 Patch objects>)

In [35]:
# Nå skaper vi vektene for den 2D matrise 
# Et histogram skal være det som gjelder bare tid
# det andre skal være det vektete flukser
#lonKanter=np.
lonKanter=np.arange(0,360,30)
latKanter=(np.arange(0,85,5)+100)/2.
latKanterSH=(np.arange(-60,25,5)-120)/2.
latKanterAll=(np.arange(-60,305,5)-120)/2.
print(lonKanter)
print(latKanter)
print(latKanterSH)
print(latKanterAll)

[  0  30  60  90 120 150 180 210 240 270 300 330]
[50.  52.5 55.  57.5 60.  62.5 65.  67.5 70.  72.5 75.  77.5 80.  82.5
 85.  87.5 90. ]
[-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50. ]
[-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90. ]


In [36]:
hist2DTid,xEdge,yEdge=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanter),weights=dts)
hist2DJe,xEdge,yEdge=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanter),weights=dts*espec.je[0])

hist2DTidSH,xEdgeSH,yEdgeSH=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanterSH),weights=dts)
hist2DJeSH,xEdgeSH,yEdgeSH=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanterSH),weights=dts*espec.je[0])

hist2DTidAll,xEdgeAll,yEdgeAll=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanterAll),weights=dts)
hist2DJeAll,xEdgeAll,yEdgeAll=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanterAll),weights=dts*espec.je[0])
hist2DNAll,xEdgeAll,yEdgeAll=np.histogram2d(mag.lon[0],mag.lat[0],bins=(lonKanter,latKanterAll))

In [37]:
histDiv = hist2DJe*0.
nzInds = np.where(hist2DTid > 0)
histDiv[nzInds] = hist2DJe[nzInds]/hist2DTid[nzInds]

histDivSH = hist2DJeSH*0.
nzIndsSH = np.where(hist2DTidSH > 0)
histDivSH[nzIndsSH] = hist2DJeSH[nzIndsSH]/hist2DTidSH[nzIndsSH]

histDivAll = hist2DJeAll*0.
nzIndsAll = np.where(hist2DTidAll > 0)
histDivAll[nzIndsAll] = hist2DJeAll[nzIndsAll]/hist2DTidAll[nzIndsAll]

In [40]:
fig2 = plt.figure(num=2,figsize=(20, 8))
ax = fig2.add_subplot(131)
ax.set_title('imshow: equidistant')
im = plt.imshow(histDiv.T, interpolation='nearest', origin='low',
                extent=[xEdge[0], xEdge[-1], yEdge[0], yEdge[-1]])
ax2 = fig2.add_subplot(132)
ax2.set_title('pcolormesh: exact bin edges')
X, Y = np.meshgrid(xEdge, yEdge)
ax2.pcolormesh(X, Y, histDiv.T)
ax2.set_aspect('auto')
# plt.show()
ax3 = fig2.add_subplot(133)
ax3.set_title('NonUniformImage: interpolated')
im3 = mpl.image.NonUniformImage(ax3, interpolation='bilinear',
                                extent=[xEdge[0], xEdge[-1], yEdge[0], yEdge[-1]])
xcenters = xEdge[:-1] + 0.5 * (xEdge[1:] - xEdge[:-1])
ycenters = yEdge[:-1] + 0.5 * (yEdge[1:] - yEdge[:-1])
im3.set_data(xcenters, ycenters, histDiv.T)
ax3.images.append(im3)
ax3.set_xlim(xEdge[0], xEdge[-1])
ax3.set_ylim(yEdge[0], yEdge[-1])
ax3.set_aspect('auto')
# fig2.colorbar(im3, ax=ax3)
plt.show()

Text(0.5,1,'imshow: equidistant')

Text(0.5,1,'pcolormesh: exact bin edges')

Text(0.5,1,'NonUniformImage: interpolated')

(0, 330)

(50.0, 90.0)

In [17]:
figSH = plt.figure(num=3,figsize=(20, 8))
axSH = figSH.add_subplot(131)
axSH.set_title('imshow: equidistant')
im = plt.imshow(histDivSH.T, interpolation='nearest', origin='low',
                extent=[xEdgeSH[0], xEdgeSH[-1], yEdgeSH[0], yEdgeSH[-1]])

axSH2 = figSH.add_subplot(132)
axSH2.set_title('pcolormesh: exact bin edges')
X, Y = np.meshgrid(xEdgeSH, yEdgeSH)
axSH2.pcolormesh(X, Y, histDivSH.T)
axSH2.set_aspect('auto')
# plt.show()
axSH3 = figSH.add_subplot(133)
axSH3.set_title('NonUniformImage: interpolated')
im3 = mpl.image.NonUniformImage(axSH3, interpolation='bilinear',
                                extent=[xEdgeSH[0], xEdgeSH[-1], yEdgeSH[0], yEdgeSH[-1]])
xcenters = xEdgeSH[:-1] + 0.5 * (xEdgeSH[1:] - xEdgeSH[:-1])
ycenters = yEdgeSH[:-1] + 0.5 * (yEdgeSH[1:] - yEdgeSH[:-1])
im3.set_data(xcenters, ycenters, histDivSH.T)
axSH3.images.append(im3)
axSH3.set_xlim(xEdgeSH[0], xEdgeSH[-1])
axSH3.set_ylim(yEdgeSH[0], yEdgeSH[-1])
axSH3.set_aspect('auto')
plt.show()

Text(0.5,1,'imshow: equidistant')

Text(0.5,1,'pcolormesh: exact bin edges')

Text(0.5,1,'NonUniformImage: interpolated')

(0, 330)

(-90.0, -50.0)

In [42]:
figAll = plt.figure(num=4,figsize=(16, 8))
# axAll = figAll.add_subplot(131)
# axAll.set_title('imshow: equidistant')
# im = plt.imshow(histDivAll.T, interpolation='nearest', origin='low',
#                 extent=[xEdgeAll[0], xEdgeAll[-1], yEdgeAll[0], yEdgeAll[-1]])

axAll2 = figAll.add_subplot(121)
axAll2.set_title('pcolormesh: exact bin edges')
X, Y = np.meshgrid(xEdgeAll, yEdgeAll)
axAll2.pcolormesh(X, Y, histDivAll.T)
axAll2.set_aspect('auto')
# plt.show()
axAll3 = figAll.add_subplot(122)
axAll3.set_title('NonUniformImage: interpolated')
im3 = mpl.image.NonUniformImage(axAll3, interpolation='bilinear',
                                extent=[xEdgeAll[0], xEdgeAll[-1], yEdgeAll[0], yEdgeAll[-1]])
xcenters = xEdgeAll[:-1] + 0.5 * (xEdgeAll[1:] - xEdgeAll[:-1])
ycenters = yEdgeAll[:-1] + 0.5 * (yEdgeAll[1:] - yEdgeAll[:-1])
im3.set_data(xcenters, ycenters, histDivAll.T)
axAll3.images.append(im3)
axAll3.set_xlim(xEdgeAll[0], xEdgeAll[-1])
axAll3.set_ylim(yEdgeAll[0], yEdgeAll[-1])
axAll3.set_aspect('auto')
figAll.colorbar(im3, ax=axAll3)
plt.show()

Text(0.5,1,'pcolormesh: exact bin edges')

Text(0.5,1,'NonUniformImage: interpolated')

(0, 330)

(-90.0, 90.0)

In [43]:
figAll2 = plt.figure(num=5,figsize=(16, 8))
# axAll = figAll.add_subplot(131)
# axAll.set_title('imshow: equidistant')
# im = plt.imshow(hist2DNAll.T, interpolation='nearest', origin='low',
#                 extent=[xEdgeAll[0], xEdgeAll[-1], yEdgeAll[0], yEdgeAll[-1]])

axAll2 = figAll2.add_subplot(121)
axAll2.set_title('pcolormesh: exact bin edges')
X, Y = np.meshgrid(xEdgeAll, yEdgeAll)
axAll2.pcolormesh(X, Y, hist2DNAll.T)
axAll2.set_aspect('auto')
# plt.show()
axAll3 = figAll2.add_subplot(122)
axAll3.set_title('NonUniformImage: interpolated')
im3 = mpl.image.NonUniformImage(axAll3, interpolation='bilinear',
                                extent=[xEdgeAll[0], xEdgeAll[-1], yEdgeAll[0], yEdgeAll[-1]])
xcenters = xEdgeAll[:-1] + 0.5 * (xEdgeAll[1:] - xEdgeAll[:-1])
ycenters = yEdgeAll[:-1] + 0.5 * (yEdgeAll[1:] - yEdgeAll[:-1])
im3.set_data(xcenters, ycenters, hist2DNAll.T)
axAll3.images.append(im3)
axAll3.set_xlim(xEdgeAll[0], xEdgeAll[-1])
axAll3.set_ylim(yEdgeAll[0], yEdgeAll[-1])
axAll3.set_aspect('auto')
figAll2.colorbar(im3, ax=axAll3)
plt.show()

Text(0.5,1,'pcolormesh: exact bin edges')

Text(0.5,1,'NonUniformImage: interpolated')

(0, 330)

(-90.0, 90.0)

In [44]:
fig1 = plt.figure(num=1,figsize=(20, 8))
ax = fig1.add_subplot(111)
plt.hist(espec.mlt[0])
plt.show()


(array([504317., 486026., 483696., 491395., 487107., 479131., 475700.,
        485037., 452047., 468933.]),
 array([21. , 21.3, 21.6, 21.9, 22.2, 22.5, 22.8, 23.1, 23.4, 23.7, 24. ],
       dtype=float32),
 <a list of 10 Patch objects>)